In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("carlosgdcj/genius-song-lyrics-with-language-information")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/genius-song-lyrics-with-language-information


In [3]:
import os
os.listdir(path)

['song_lyrics.csv']

In [14]:
import pandas as pd
data = pd.read_csv(path+"/song_lyrics.csv", skiprows=lambda x: x % 3 != 0)
data.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
1,Lollipop Remix,rap,Lil Wayne,2008,580832,"{""Kanye West"",""Static Major""}",[Intro: Lil Wayne]\nHaha\nUh-huh\nNo homo (You...,7,en,en,en
2,Rockin and Rollin,rap,Cam'ron,1998,6399,"{""Cam\\'ron""}",[Verse 1]\nAy yo you wonder who I are\nI guzzl...,10,en,en,en
3,Think Yall Know,rap,Fabolous,2003,2530,{},[Verse 1]\nYou ain't never seen a nigga this l...,13,en,en,en
4,98 Freestyle,rap,Big L,2000,297788,{},"[Verse 1]\nYo, fuck all the glamours and glitz...",16,en,en,en


In [15]:
data

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
1,Lollipop Remix,rap,Lil Wayne,2008,580832,"{""Kanye West"",""Static Major""}",[Intro: Lil Wayne]\nHaha\nUh-huh\nNo homo (You...,7,en,en,en
2,Rockin and Rollin,rap,Cam'ron,1998,6399,"{""Cam\\'ron""}",[Verse 1]\nAy yo you wonder who I are\nI guzzl...,10,en,en,en
3,Think Yall Know,rap,Fabolous,2003,2530,{},[Verse 1]\nYou ain't never seen a nigga this l...,13,en,en,en
4,98 Freestyle,rap,Big L,2000,297788,{},"[Verse 1]\nYo, fuck all the glamours and glitz...",16,en,en,en
...,...,...,...,...,...,...,...,...,...,...,...
1711613,Cut,rap,Serane,2022,3,{},"[Paroles de ""Cut""]\n\n[Intro]\nSurf Gang\nYeah...",7882827,fr,fr,fr
1711614,Baby Hold On,pop,Ryan Egan,2022,5,{},At this crossroads\nI don’t wanna go and gambl...,7882831,en,en,en
1711615,Everything Is Alright Now,pop,Chuck Bernard,2013,2,{},"Everything is alright now\nOh yes, baby\nEvery...",7882838,en,en,en
1711616,Un ja fali,rap,Endi & Ilias,2021,3,{},Ahh\nNhhh\nHhhh\nUn ja fali\nA tka marr malli\...,7882841,de,de,de


In [16]:
data[data['artist'] == 'Taylor Swift']

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
19395,Safe Sound,pop,Taylor Swift,2012,238499,"{""The Civil Wars""}",[Verse 1: Taylor Swift]\nI remember tears stre...,62236,en,en,en
25606,Innocent,country,Taylor Swift,2010,145880,{},[Verse 1]\nI guess you really did it this time...,83166,en,en,en
26111,We Are Never Ever Getting Back Together,pop,Taylor Swift,2012,353358,{},[Verse 1]\nI remember when we broke up the fir...,84880,en,en,en
28083,I Knew You Were Trouble.,pop,Taylor Swift,2012,353452,{},"[Intro]\n\n[Verse 1]\nOnce upon a time, a few ...",92224,en,en,en
29499,Red,pop,Taylor Swift,2012,257927,{},[Verse 1]\nLoving him is like driving a new Ma...,96342,en,en,en
...,...,...,...,...,...,...,...,...,...,...,...
1602398,Sweeter Than Fiction Taylors Version,country,Taylor Swift,2021,56,{},"[Verse 1]\nHit the ground, hit the ground, hit...",7320790,en,en,en
1609406,Mine Pop Mix,pop,Taylor Swift,2011,55,{},"[Verse 1]\nYou were in college, working part-t...",7354339,en,en,en
1617065,Red Taylors Version Traducción asturiana,pop,Taylor Swift,2021,22,{},(PRIMER VERSU)\n\nAmalu ye como\nConducir un M...,7390793,es,ast,NaN
1654960,magic ball,pop,Taylor Swift,2022,28,{},[Verse 1]\nIt's weird how all of you guys are ...,7579769,en,en,en


In [17]:
def simple_search(query):
    return data[data['lyrics'].str.contains(query)]

In [22]:
simple_search("Stacy's mom has got it goin' on")

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
494035,Stacys Mom,pop,Bowling for Soup,2011,16029,{},[Intro]\nStacy's mom has got it goin' on\nStac...,1850115,en,en,en
1325800,Where Is My Mom,rock,Neil Cicierega,2020,3976,{},"[Intro]\nStop\n\n[Verse 1]\nStacy, can I come ...",6013693,en,en,en


Steps:

1. lowercase everything
2. remove punctuation
3. not sure if lemmatization/stemming is useful since we'd often want exact matches on lyrics

In [25]:
from string import punctuation

def process(lyric):
  processed_lyric = lyric.lower()
  processed_lyric = processed_lyric.translate(str.maketrans('', '', punctuation))
  return processed_lyric

In [26]:
data['processed'] = data['lyrics'].apply(process)

In [27]:
def simple_search_cleaned(query):
  processed_query = process(query)
  return data[data['processed'].str.contains(processed_query)]

In [30]:
simple_search_cleaned("take your time coming home")

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,processed
78427,Take Your Time Coming Home,rock,fun.,2009,11925,{​fun.},[Intro]\nDin-an-ah\nDin-an-ah\nHoldin' on-nah\...,296937,en,en,en,intro\ndinanah\ndinanah\nholdin onnah\nholdin ...
209191,Take Your Time,pop,fun.,2015,1099,{​fun.},Take your time coming home\nHear the wheels as...,934454,en,en,en,take your time coming home\nhear the wheels as...
365458,Keep from Moving,pop,Creeper Lagoon,2001,40,{},Take down this ragged nest\nYou have built on ...,1427161,en,en,en,take down this ragged nest\nyou have built on ...
